In [1]:
SEED = 11

In [2]:
import torch
import matplotlib.pyplot as plt
from IPython.display import display, Math

In [3]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader

In [4]:
batch_size = 256
learning_rate = 0.0002
num_epoch = 10

## mnist data load

In [9]:
mnist_train = datasets.MNIST("./", train=True,
                            transform = transforms.ToTensor(),
                            target_transform=None,
                            download=True)

In [10]:
mnist_test = datasets.MNIST("./", train=False,
                            transform = transforms.ToTensor(),
                            target_transform=None,
                            download=True)

In [11]:
train_loader = DataLoader(mnist_train, batch_size = batch_size,
                         shuffle = True, num_workers = 2,
                         drop_last = True)

In [12]:
test_loader = DataLoader(mnist_test, batch_size = batch_size,
                         shuffle = False, num_workers = 2,
                         drop_last = True)

In [37]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.layer = nn.Sequential(
            nn.Conv2d(in_channels = 1,
                      out_channels = 16,
                      kernel_size = 5, 
                      stride = 1,
                      padding = 0,
                      ),
            nn.ReLU(),
            nn.Conv2d(16, 32, 5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2,
                         stride = 2),
            nn.Conv2d(32, 64, 5),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
            # [None, 64, 3, 3]
        )
        self.fc_layer = nn.Sequential(
            nn.Linear(64*3*3, 100),
            nn.ReLU(),
            nn.Linear(100, 10),
        )
    def forward(self, x):
        out = self.layer(x)
        out = out.view(batch_size, -1)
        out = self.fc_layer(out)
        return(out)
        

In [38]:
device = torch.device("cuda:0")
model = CNN().to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [47]:
arr_loss = []
for ii in range(num_epoch):
    for jj, [image, label] in enumerate(train_loader):
        x = image.to(device)
        y_ = label.to(device)
        
        optimizer.zero_grad()
        output = model.forward(x)
        loss = loss_func(output, y_)
        loss.backward()
        optimizer.step()
        
        if jj % 1000 == 0:
            print(loss)
            arr_loss.append(loss.cpu().detach().numpy())

tensor(0.1405, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.1121, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.1256, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0675, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0706, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.1407, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0760, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0566, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0735, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0738, device='cuda:0', grad_fn=<NllLossBackward>)


In [65]:
correct = 0
total = 0

with torch.no_grad():
    for image, label in test_loader:
        x = image.to(device)
        y_ = label.to(device)
        output = model.forward(x)
        _, output_index = torch.max(output, 1)
        total += label.size(0)
        correct += (output_index == y_).sum().float()
    print('Test acc: {}'.format(correct/total))

Test acc: 0.9803686141967773
